In [2]:
!pip install transformers --quiet
!pip install fasttext --quiet

     |████████████████████████████████| 1.4MB 11.3MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
     |████████████████████████████████| 2.9MB 55.2MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 


In [3]:
from nltk.corpus import brown
import nltk

import string
import os, errno

import fasttext
import csv

import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchtext.data import Field, TabularDataset, BucketIterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [5]:
# Download the Brown corpus
nltk.download('brown')

# Assert that necessary Python files are in current directory
assert os.path.exists('./createModel.py')
assert os.path.exists('./demoBERT.py')
#assert os.path.exists('./demoRoBERTa.py')

# Create a 'lstm' folder if it doesn't exist
try:
  os.makedirs('lstm')
except OSError as e:
  if e.errno != errno.EEXIST:
    raise

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [25]:
from createModel import *
#from demoBERT import *
from demoRoBERTa import *

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ranks before : [1783, 15334, 1255, 6336, 9214, 10334, 3409, 261, 1622, 52]
cover_text:  Hello world, today we are going to present you a tutorial about how to cook a turkey. Please follow this tutorial carefully,  suspKKpe replied threatens shootingnick […] Example and

ranks after: [1783, 15334, 1255, 6336, 9214, 10334, 3409, 261, 1622, 52]
. this message is super secret please do not share it


In [26]:
mod, tok = buildModelRoBERTa()

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def gen_fake_text(start_text, start_secret, secret_text):
  ranks = get_ranks(mod, tok, start_secret, secret_text)
  cover_text = generate_cover_text(mod, tok, start_text, ranks)

  output_text = cover_text
  return output_text

In [28]:
start_text = """
Welcome to the Shakespeare festival in London, UK. Today
"""

start_secret = "Secret"
secret_text = "A secret message for Joe"

In [29]:
print(gen_fake_text(start_text, start_secret, secret_text))


Welcome to the Shakespeare festival in London, UK. Today
YouadjKickrov Condition


In [30]:
news_text = brown.sents(categories=['news'])[:100]
fake_news = []
true_news = []

secret_text_length = 5

In [31]:
for sentence in news_text:
  if len(sentence) <= secret_text_length:
    continue

  sentence_true = ' '.join(sentence)

  sentence_true = sentence_true.lower()

  true_news.append(sentence_true)

  sentence_fake = ' '.join(sentence[:-secret_text_length])
  print(sentence_fake)
  sentence_fake = gen_fake_text(sentence_fake, start_secret, secret_text)

  fake_news.append(sentence_fake)

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which
The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by
`` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size
The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate
It recommended that Fulton legislators act `` to have these laws studied and revised to the end of modernizing
The grand jury commented on a number of other topics , among them the Atlanta and

In [32]:
punctuation_dict = {}
for x in string.punctuation:
  punctuation_dict[ord(x)] = None

print(true_news[1].translate(punctuation_dict))

for i in range(len(true_news)):
  true_news[i] = true_news[i].translate(punctuation_dict)

for i in range(len(fake_news)):
  fake_news[i] = fake_news[i].translate(punctuation_dict)

# s.translate({ord(c): None for c in string.whitespace})

the jury further said in termend presentments that the city executive committee  which had overall charge of the election   deserves the praise and thanks of the city of atlanta  for the manner in which the election was conducted 


In [33]:
for i in range(len(true_news)):
  true_news[i] = true_news[i].lower()

In [34]:
for i in range(len(true_news)):
  true_news[i] = true_news[i].strip()

for i in range(len(fake_news)):
  fake_news[i] = fake_news[i].strip()

In [35]:
destination_folder = './lstm/'

train_test_ratio = 0.90
train_valid_ratio = 0.80

first_n_words = 200

def trim_string(x):
    x = x.split(maxsplit=first_n_words)
    x = ' '.join(x[:first_n_words])
    return x

In [36]:
all_dict = {'text': true_news + fake_news, 'label': [0]*len(true_news)+[1]*len(fake_news)}

df = pd.DataFrame(data=all_dict, columns=['label', 'text'])
df

,label,text
0,0,the fulton county grand jury said friday an in...
1,0,the jury further said in termend presentments ...
2,0,the septemberoctober term jury had been charge...
3,0,only a relative handful of such reports was re...
4,0,the jury said it did find that many of georgia...
...,...,...
179,1,Many local citizens feared that there would be...
180,1,Sheriff Felix Tabb said the ordinary apparentl...
181,1,Everything went real smooth Security banner li...
182,1,There wasnt a signing bustling bast winterPatrick


In [37]:

df['label'] = df['label'].astype('str')
df['label'] = '__label__' + df['label']

In [38]:

df_fasttext_train, df_fasttext_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)

In [39]:
df_fasttext_train.to_csv('train.txt', 
          index = False, 
          sep = ' ',
          header = None, 
          quoting = csv.QUOTE_NONE, 
          quotechar = "", 
escapechar = " ")

df_fasttext_test.to_csv('test.txt', 
          index = False, 
          sep = ' ',
          header = None, 
          quoting = csv.QUOTE_NONE, 
          quotechar = "", 
escapechar = " ")


In [40]:
model = fasttext.train_supervised('train.txt', wordNgrams = 10)

In [41]:
model.test('test.txt')

(19, 0.5263157894736842, 0.5263157894736842)